In [ ]:
from pyknp import KNP

In [ ]:
# 定義
file_info = "./data/input/RS_data/rsdata" # アノテーションデータのPATH
corpus_file = "corpus_5w1hs_ch6.txt" # 出力コーパスの名前

In [ ]:
def get_iobcorpus(file_name):
    
    # アノテーション情報用意
    file_ann = "{0}.ann".format(file_name)
    tag_number = [] # アノテーションラベルの先頭番号と末尾番号リスト
    tag_vocab = {} # アノテーションラベルの名前と先頭番号
    with open(file_ann, 'r') as f:
        for i in f.read().split("\n"):
            if len(i) != 0:
                t_s = int(i.split()[2]) # アノテーションの先頭
                t_e = int(i.split()[3]) # アノテーションの末尾
                tag_vocab[t_s] = i.split()[1]
                tag_number.append([i for i in range(t_s,t_e)])
        tag_number = sorted(tag_number,key=lambda x: x[0])
    
    # コーパスの作成
    file_txt = "{0}.txt".format(file_name)

    wc_end = 0 # 記事の単語サイズ
    features_set = [] # 記事の特徴量リスト
    
    n = 0 # タグ番号のインデックス
    p = -1 # アノテーションのラベルのインデックス
        
    knp = KNP()
    with open(file_txt, 'r') as f:
        data_text = f.read() # ファイル読み込み
        
        # 1記事を1行に分けて処理する
        text_line = data_text.split("。")        
        for text in text_line:
            if len(text) == 0:
                continue
            text += "。"
            
            text_dic = {} # 単語と表層形の情報を保存 (key：形態素番号(link), value：単語番号,表層系)
            tdic_key = 0 # 形態素番号
            
            bnst_info = {} #文節の情報を保存 (key：文節id, value：親id, 形態素番号(link)リスト)
            
            # 文節ごとの処理
            result = knp.parse(text)    
            for bnst in result.bnst_list(): 
                mrph_list = bnst.mrph_list() 
                
                bi_value = {} # bnst_infoの中身
                mrph_num_list = [] # 形態素番号(link)リスト
                
                # 形態素ごとの処理
                for mrph in mrph_list: 

                    tdic_value = {} # text_dicの中身
                    
                    word_count = [wc_end + i for i in range(len(mrph.midasi))] # 単語サイズ計算
                    wc_end += len(mrph.midasi)
                    
                    # text_dicの格納
                    tdic_value["wc_info"] = word_count                    
                    tdic_value["suff"] = mrph.midasi
                    tdic_value["hinsi"] = mrph.hinsi
                    tdic_value["bunrui"] = mrph.bunrui
                    text_dic[tdic_key] = tdic_value
                    
                    # 形態素番号(link)リストの格納
                    mrph_num_list.append(tdic_key)
                    
                    tdic_key += 1
                
                # bnst_infoの格納
                bi_value["parent_id"] = bnst.parent_id 
                bi_value["mrph_id"] = mrph_num_list
                bnst_info[bnst.bnst_id] = bi_value
                
            bnst_length_dic = get_bnstlen(bnst_info) # 形態素の文末からの係り受けの長さを取得
                        
            # アノテーションの付与・特徴データの作成
            for k,v in sorted(text_dic.items()):
                
                features = [] # 特徴
                
                word_count = v["wc_info"]
                bnst_length = bnst_length_dic[k]
                features.extend([v["suff"], v["hinsi"], v["bunrui"], str(bnst_length)])
                
                # 文末処理
                if n >= len(tag_number):
                    features.append("O") 
                    features_set.append(features)
                    continue 
                    
                # タグ処理 
                match_n = list(set(tag_number[n]) & set(word_count))
                
                if len(match_n) > 0: # ラベルあり                        
                    if tag_number[n][0] in word_count and tag_number[n][-1] in word_count: # 単体
                        p = tag_number[n][0] # ラベル情報を設定
                        features.append("B-{0}".format(tag_vocab[p]))
                        n += 1 # 対象とするラベル情報の変更
                            
                    else: # 継続あり                            
                        if tag_number[n][0] == word_count[0]: # B 
                            p = tag_number[n][0] # ラベル情報を設定
                            features.append("B-{0}".format(tag_vocab[p]))
                                
                        elif tag_number[n][-1] in word_count: # ラベルの最後
                            features.append("I-{0}".format(tag_vocab[p]))
                            n += 1 # 対象とするラベル情報の変更
                                
                        else: # I
                            features.append("I-{0}".format(tag_vocab[p]))
                                
                else: # ラベルなし
                    features.append("O")
                        
                features_set.append(features)
                 
    return features_set

In [ ]:
"""
文末からの係り受けの長さを取得する関数
入力：文節の情報辞書 (key：形態素番号(link), value：単語番号,表層系)
出力：文末からの長さ辞書 (key：形態素id, value：文末からの長さ)
"""
def get_bnstlen(bnst_info):
    
    bnst_length_dic = {}
    
    # 1. 全ての文節の文末からの長さを取得する
    for my_id in bnst_info.keys():
        line_length = 0
        
        flag = True
        serch_id = my_id

        # 1-1. 文節ごとに、最後の係り先になるまでループする
        while flag:
            parent_id = bnst_info[serch_id]["parent_id"]

            if parent_id != -1: # 係り先が最後じゃないとき、長さをインクリメントする
                line_length += 1
                
            else: # 係り先が最後のとき、ループを抜ける
                flag = False
                
            serch_id = parent_id # 1-2.係り先のidを次の探索idとする
        
        for mrph_id in bnst_info[my_id]["mrph_id"]:
            bnst_length_dic[mrph_id] = line_length # 1-3.文節ごとに、係り受け順を格納
            
    return bnst_length_dic

In [ ]:
# 通常用
data_size = 215 # データサイズ
skip_file = [107,129,125,151,173,195]
data_sum = 0

# 書き出しファイル準備
output_f = open(corpus_file, "w")

for file_id in range(1,data_size+1): 

    if file_id in skip_file:
        continue
        
    data_sum += 1
    print(file_id,end="")
    file_name = "{0}{1}".format(file_info,file_id)
    for line in get_iobcorpus(file_name):
        output_line = "\t".join(line)
        output_f.writelines(output_line+"\n")
    output_f.writelines("\n") # 5w1h用の改行コマンド
    
output_f.close()

print("")
print(data_sum)